In [ ]:
import hist_igcs, viewer_util, perturbation
from moleculetda.read_file import read_data
import ipywidgets
import nglview as nv
import os, torch,json, glob,pickle
from pymatgen.transformations.advanced_transformations import CubicSupercellTransformation
from pymatgen.core import Molecule, Structure
import pandas as pd
import numpy as np

In [ ]:
os.makedirs('tobacco_pdgm', exist_ok=True)
os.makedirs('perturb4921_dgm', exist_ok=True)

In [ ]:
k=4921;filename = f'tobacco_cif/tobmof-{k}.cif'

In [ ]:
#for _ in tqdm.tqdm(range(584)):
#    dumpname = f'perturb4921/{perturbation.randomname(8)}.pkl'
#    perturbation.read_and_perturb_cif(filename=filename, dumpname=dumpname)

#complete reproduce of this perturbation needs to modify pymatgen

In [ ]:
structure = Structure.from_file(filename)
dumpname = f'perturb4921/original_str.pkl'
with open(dumpname, mode="wb") as f:
        pickle.dump(structure, f)

In [ ]:
perturbation.dgm_dump(pkl_suffix = "perturb4921/????????.pkl", dgm_dir = "perturb4921_dgm")

In [ ]:
perturbation.construct_hist_unit_transnegative(
    original_structure_pkl=dumpname,
    pkl_suffix="perturb4921/????????.pkl",  # perturbed file lists
    output_feat_df_fn = "perturb4921/bins_perturb_unit36_max72.csv",  # target
    unit = 36,
    max_size = 72,
    cutoff = False,
)

In [ ]:
perturbation.dgm_to_landscape(    original_dgm='perturb4921_dgm/original_str.pkl_dgm.pkl',
    dgm_prefix='perturb4921_dgm/????????.pkl_dgm.pkl',
    output_npy_fn='perturb4921/stacked_x.npy',
    px_size=54,
    onemaxB = 27.0,
    twomaxB = 27.0,
    onemaxP = 8.8,
    twomaxP = 3.5,)

In [ ]:
model_fn = 'model_tobacco.pkl'
with open(model_fn, mode="rb") as f:
    rf = pickle.load(f)

x = np.load('perturb4921/stacked_x.npy')
y = rf.predict(x.reshape(x.shape[0], -1))
np.save('perturb4921/stacked_y.npy', y)

In [ ]:
perturbation.igcs_total_single(
    feat_df_fn="perturb4921/bins_perturb_unit36_max72.csv",
    target_fn='perturb4921/stacked_y.npy',  # made from x above
    output_igcs_npz_fn="perturb4921/igcs_direct_single",
    ratio=0.01 #small enough to distinguish numbers of atoms in grids
)

In [ ]:
ps = viewer_util.ViewerUtil(cif_fn=f'tobacco_cif/tobmof-{k}.cif', pdgm_fn=f'tobacco_pdgm/tobmof-{k}.pdgm')
bdy147 = ps.bdy_symbols(dim=1, pair_id=147, reduce=False)
bdy149 = ps.bdy_symbols(dim=1, pair_id=149, reduce=False)

In [ ]:
snap_predict = np.load(f'perturb{k}/igcs_direct_single_bins_perturb_unit36_max72_0.npy')
df = pd.read_csv(f'perturb{k}/bins_perturb_unit36_max72.csv').set_index("Unnamed: 0")
ser = df.iloc[0]
serr = pd.Series(snap_predict, index=ser.index)
unit=36
max_size=72
divided_full = dict()
for i in range(ps.coords.shape[0]):
    coords = ps.coords
    for j in range(3):
        trans = np.min(coords, axis=0)[j]
        if trans < 0:
            # print(coords)
            coords[:, j] -= trans
    x, y, z = (coords[i] * unit / max_size).astype(int) / unit * max_size 
    if f"{x}-{y}-{z}" not in serr.index: print(x,y,z)
    contrib = serr[f"{x}-{y}-{z}"] / ser[f"{x}-{y}-{z}"]
    divided_full[i] = contrib
ratio_ser = pd.Series(divided_full, index=divided_full.keys())

In [ ]:
void_in_list=list()
for l in serr[ser==0].index:
    void_in_list.append(l)

Figure 3(a)

In [ ]:
nview_2403 = nv.show_pymatgen(Structure.from_file(f'tobacco_cif/tobmof-4921.cif'))
nview_2403.add_unitcell()
nview_2403.camera = "perspective"
nview_2403

Figure 3(b)

In [ ]:
darkgrey = [ 0, 0.9, 0.9 ]
yellow = [0.9, 0.9,0 ]

nview_2403 = nv.show_pymatgen(Structure.from_file(f'tobacco_cif/tobmof-{k}.cif'))
nview_2403.add_unitcell()
nview_2403.camera = "perspective"
for i in range(ps.coords[bdy149].shape[0]):
    #print(i)
    j = bdy149[i] #enumerate
    nview_2403.shape.add_sphere(ps.coords[bdy149][i],\
                                list(np.array(darkgrey)*(ratio_ser[j]-np.min(ratio_ser[bdy149]))/(np.max(ratio_ser[bdy149].values)-np.min(ratio_ser[bdy149]))) ,\
                                ps.pd1_df.loc[149].birth_sqrt)

nview_2403

Figure 3(c)

In [ ]:
darkgrey = [ 0, 0.9, 0.9 ]
yellow = [0.9, 0.9,0 ]

nview_2403 = nv.show_pymatgen(Structure.from_file(f'tobacco_cif/tobmof-{k}.cif'))
nview_2403.add_unitcell()
nview_2403.camera = "perspective"
for i in range(ps.coords[bdy147].shape[0]):
    #print(i)
    j = bdy147[i] #enumerate
    nview_2403.shape.add_sphere(ps.coords[bdy147][i],\
                                list(np.array(darkgrey)*(ratio_ser[j]-np.min(ratio_ser[bdy147]))/(np.max(ratio_ser[bdy147].values)-np.min(ratio_ser[bdy147]))) ,\
                                ps.pd1_df.loc[147].birth_sqrt)

nview_2403

In [ ]:
def mesh_list(x,y,z):
    return list(np.hstack([list(np.array([x,y,z])),\
                                     list(np.array([x+2,y,z])), \
                                     list(np.array([x,y+2,z])),\
                                     list(np.array([x+2,y,z])),\
                                     list(np.array([x+2,y+2,z])),\
                                     list(np.array([x+2,y,z+2])), \
                                     list(np.array([x+2,y+2,z+2])),\
                                     list(np.array([x+2,y,z+2])),\
                                     list(np.array([x,y+2,z+2])), \
                                                                         list(np.array([x,y,z])),\
                                     list(np.array([x,y+2,z])),\
                                     list(np.array([x,y,z+2])), \
                                                                                                             list(np.array([x,y,z])),\
                                     list(np.array([x+2,y,z])),\
                                     list(np.array([x,y,z+2])), \
                                       list(np.array([x+2,y+2,z+2])),\
                                     list(np.array([x+2,y+2,z])),\
                                     list(np.array([x,y+2,z+2])), \
                                       list(np.array([x,y+2,z+2])),\
                                     list(np.array([x,y,z+2])),\
                                     list(np.array([x,y+2,z])), \
                                       list(np.array([x,y+2,z])),\
                                     list(np.array([x,y+2,z+2])),\
                                     list(np.array([x+2,y+2,z])), \
                                       list(np.array([x+2,y,z+2])),\
                                     list(np.array([x+2,y,z])),\
                                     list(np.array([x,y,z+2])), \
                                       list(np.array([x+2,y+2,z])),\
                                     list(np.array([x+2,y,z])),\
                                     list(np.array([x,y+2,z])), \
                                       list(np.array([x,y,z+2])),\
                                     list(np.array([x+2,y,z+2])),\
                                     list(np.array([x,y+2,z+2])), \
                                    
                                    list(np.array([x+2,y+2,z+2])),\
                                     list(np.array([x+2,y+2,z])), \
                                     list(np.array([x+2,y,z+2]))]))

Figure 3(d)

In [ ]:
yellow = [0.9, 0,0.9 ]

nview_2403 = nv.show_pymatgen(Structure.from_file(f'tobacco_cif/tobmof-4921.cif'))
nview_2403.add_unitcell()
nview_2403.camera = "perspective"
for i,l in enumerate(list(serr[void_in_list].sort_values(ascending=False).tail(5).index)):
    x,y,z=l.split('-')
    x=float(x)
    y=float(y)
    z=float(z)
    value = serr[l] #item
    nview_2403.shape.add_mesh(mesh_list(x,y,z), [0.5 for i in range(108)])
    #nview_2403.shape.add_text([x,y,z], [0,0,0], 5, f'{np.round(value*100,2)}'[:5])
nview_2403

Figure 3(e)

In [ ]:
yellow = [0.9, 0,0.9 ]

nview_2403 = nv.show_pymatgen(Structure.from_file(f'tobacco_cif/tobmof-4921.cif'))
nview_2403.add_unitcell()
nview_2403.camera = "perspective"
for i,l in enumerate(list(serr[void_in_list].sort_values(ascending=False).head(5).index)):
    x,y,z=l.split('-')
    x=float(x)
    y=float(y)
    z=float(z)
    value = serr[l] #item
    nview_2403.shape.add_mesh(mesh_list(x,y,z), [0.5 for i in range(108)])
    #nview_2403.shape.add_text([x,y,z], [0,0,0], 5, f'{np.round(value*100,2)}'[:5])
nview_2403

In [ ]:
ratio_ser.describe()